In [1]:
import time
start_time = time.time()

# Preparation - Load the packages and the data

Import the packages and load the data

In [2]:
# Import needed packages
import os
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import KBinsDiscretizer
import seaborn as sns; sns.set()
import glob
from sklearn.linear_model import LinearRegression
from statsmodels.formula.api import ols
import re
import nltk
from langdetect import detect
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing
from lime import lime_text
import gensim
import gensim.downloader as gensim_api
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K
import transformers
plt.style.use('seaborn-whitegrid')
sns.set_style("whitegrid")
from bs4 import BeautifulSoup
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.snowball import EnglishStemmer
import spacy
from spacy import displacy
from wordcloud import WordCloud
from sklearn.preprocessing import KBinsDiscretizer
from nbpep8.nbpep8 import pep8
from sklearn.metrics.pairwise import cosine_similarity
from textstat import textstat
from tqdm import tqdm

In [3]:
#Set the working directory
os.chdir('C:\\Users\\piotr\\OneDrive - Erasmus University Rotterdam\\Thesis\\')
pd.set_option('display.max_columns', None)

In [4]:
#Load the merged_df from the directory 'C:\\Users\\piotr\\OneDrive - Erasmus University Rotterdam\\Thesis\\Master Thesis Data\\', save index column as 'Index':
merged_df = pd.read_csv('Master Thesis Data\\df_clean_transformed.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'Master Thesis Data\\df_clean_transformed.csv'

In [ ]:
#Save index of merged_df as 'Index' column:
merged_df['Index'] = merged_df.index

In [ ]:
non_lemmatized_df = pd.read_csv('Master Thesis Data\\text_df_clean_2.csv')

In [ ]:
#Rename column 'Post_Body' to 'Post_Body_EN':
non_lemmatized_df.rename(columns={'Post_Body': 'Post_Body_EN'}, inplace=True)

In [ ]:
#Left join of merged_df and non_lemmatized_df left on 'Index' right on 'Unnamed: 0':
merged_df_2 = pd.merge(merged_df, non_lemmatized_df, how='left', left_on='Index', right_on='Unnamed: 0')

In [ ]:
merged_df_2.head()

,Post_Creation_Date,Post_View_Count,Post_Title,Post_Score,Post_Answer_Count,Response_Scores,Post_Author,Answer_Author,Forum,Week_of_Year,tag_1,tag_2,tag_3,Post_Body_clean,tag_combined,Weekly_Post_Count,D,T,Title_Length,Body_Length,Post_Body,Index,Unnamed: 0,Post_Body_EN
0,2021-07-01 00:00:58,5867,"railsinstaller.org website was not found, what...",1,3,1,8530236,8530236,Stack Overflow,26,ruby-on-rails,ssl,NaN,im taking course ruby rail video instructor re...,ruby-on-railsssl,30616,0,0,59,315,<p>I'm taking a course on Ruby on Rails. In th...,0,0.0,I'm taking a course on Ruby on Rails. In the v...
1,2021-07-01 00:01:10,11,Insert page break if MS Word document is endin...,0,0,0,3893947,0,Stack Overflow,26,ms-word,NaN,NaN,erp system generates statement via word docx t...,ms-word,30616,0,0,65,718,<p>I have an ERP system which generates statem...,1,1.0,I have an ERP system which generates statement...
2,2021-07-01 00:01:18,1393,Plotly: How to show subplot titles at the bottom,1,1,2,2313889,13107804,Stack Overflow,26,python,plotly,subplot,given example plotly documentation displayed c...,plotlypython,30616,0,0,48,1095,"<p>Given this example from <a href=""https://pl...",2,2.0,Given this example from plotly documentation\n...
3,2021-07-01 00:01:28,660,Javascript files not loading in HTML page,1,2,1,15301811,4014834,Stack Overflow,26,javascript,html,d3.js,folder structure like referenced datajs appjs ...,htmljavascript,30616,0,0,41,7146,<p>I have a folder structure like this:</p>\n<...,3,3.0,I have a folder structure like this:\n\nI have...
4,2021-07-01 00:02:36,59,How can I put a fragment inside of a tab?,0,2,2,14595783,1440565,Stack Overflow,26,java,android,xml,android studio project fragment created displa...,androidjava,30616,0,0,41,12859,<p>In my Android Studio project I have a fragm...,4,4.0,In my Android Studio project I have a fragment...


Uncomment for the final run

In [ ]:
df = merged_df_2.copy()

Temporary - create a sample!


In [ ]:
# #df is a sample of 100000 rows from the merged_df
# df = merged_df_2.sample(100000, random_state=42)

# 5. Readability of threads 

In [ ]:
df.shape

(3040730, 24)

In [ ]:
print(df.columns)

Index(['Post_Creation_Date', 'Post_View_Count', 'Post_Title', 'Post_Score',
       'Post_Answer_Count', 'Response_Scores', 'Post_Author', 'Answer_Author',
       'Forum', 'Week_of_Year', 'tag_1', 'tag_2', 'tag_3', 'Post_Body_clean',
       'tag_combined', 'Weekly_Post_Count', 'D', 'T', 'Title_Length',
       'Body_Length', 'Post_Body', 'Index', 'Unnamed: 0', 'Post_Body_EN'],
      dtype='object')


In [ ]:
# #df drop Index and Unnamed: 0 column:
# df.drop(['Index', 'Unnamed: 0'], axis=1, inplace=True)


In [ ]:
df

,Post_Creation_Date,Post_View_Count,Post_Title,Post_Score,Post_Answer_Count,Response_Scores,Post_Author,Answer_Author,Forum,Week_of_Year,tag_1,tag_2,tag_3,Post_Body_clean,tag_combined,Weekly_Post_Count,D,T,Title_Length,Body_Length,Post_Body,Post_Body_EN
0,2021-07-01 00:00:58,5867,"railsinstaller.org website was not found, what...",1,3,1,8530236,8530236,Stack Overflow,26,ruby-on-rails,ssl,NaN,im taking course ruby rail video instructor re...,ruby-on-railsssl,30616,0,0,59,315,<p>I'm taking a course on Ruby on Rails. In th...,I'm taking a course on Ruby on Rails. In the v...
1,2021-07-01 00:01:10,11,Insert page break if MS Word document is endin...,0,0,0,3893947,0,Stack Overflow,26,ms-word,NaN,NaN,erp system generates statement via word docx t...,ms-word,30616,0,0,65,718,<p>I have an ERP system which generates statem...,I have an ERP system which generates statement...
2,2021-07-01 00:01:18,1393,Plotly: How to show subplot titles at the bottom,1,1,2,2313889,13107804,Stack Overflow,26,python,plotly,subplot,given example plotly documentation displayed c...,plotlypython,30616,0,0,48,1095,"<p>Given this example from <a href=""https://pl...",Given this example from plotly documentation\n...
3,2021-07-01 00:01:28,660,Javascript files not loading in HTML page,1,2,1,15301811,4014834,Stack Overflow,26,javascript,html,d3.js,folder structure like referenced datajs appjs ...,htmljavascript,30616,0,0,41,7146,<p>I have a folder structure like this:</p>\n<...,I have a folder structure like this:\n\nI have...
4,2021-07-01 00:02:36,59,How can I put a fragment inside of a tab?,0,2,2,14595783,1440565,Stack Overflow,26,java,android,xml,android studio project fragment created displa...,androidjava,30616,0,0,41,12859,<p>In my Android Studio project I have a fragm...,In my Android Studio project I have a fragment...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3040725,2023-04-30 16:46:47,3,How do the LT and GT opcodes differ from SLT a...,0,1,0,116681,0,Ethereum,17,solidity,evm,opcode,reading opcodes evm article saw opcodes evalua...,evmsolidity,62331,1,1,66,441,"<p>When reading the <a href=""https://ethereum....","When reading the Opcodes for the EVM article, ..."
3040726,2023-04-30 17:21:18,15,is it possible to give metamask a raw transact...,0,0,0,115531,0,Ethereum,17,metamask,web3.py,NaN,building dapp based microservice testing somet...,metamaskweb3.py,62331,1,1,83,1314,<p>am building Dapp based on microservice and ...,am building Dapp based on microservice and am ...
3040727,2023-04-30 21:02:45,41,Sending ETH to Gnosis Safe,0,0,0,98043,0,Ethereum,17,gnosis-safe,NaN,NaN,contract deployed already eth autosent wallet ...,gnosis-safe,62331,1,1,26,757,"<p>so have a contract deployed already, and ET...","so have a contract deployed already, and ETH i..."
3040728,2023-04-30 22:03:50,11,How to override a function when two implementi...,0,1,0,37215,0,Ethereum,17,function-override,NaN,NaN,two contract implement one marked marked creat...,function-override,62331,1,1,91,1256,<p>I have two contracts that implement <code>s...,I have two contracts that implement . One is ...


In [ ]:
# dfr = {k: v for k, v in df.groupby('tag_combined')}

In [ ]:
import re

def calculate_lix_text(text):
    words = text.split()
    word_count = len(words)
    long_words = len([w for w in words if len(w) > 6])
    period_count = len(re.findall(r'\.', text))

    if period_count == 0 or word_count == 0:  # to avoid division by zero
        return float('NaN')

    lix = word_count / period_count + float(100 * long_words) / word_count
    return lix

def calculate_rix_text(text):
    words = text.split()
    long_words = len([w for w in words if len(w) > 6])
    period_count = len(re.findall(r'\.', text))

    if period_count == 0:  # to avoid division by zero
        return float('NaN')

    rix = long_words / period_count
    return rix



In [ ]:
def calculate_ari(df):
    df['Post_Body_EN'] = df['Post_Body_EN'].fillna('').astype(str)
    df['ari'] = df['Post_Body_EN'].apply(textstat.automated_readability_index)
    return df

def calculate_fre(df):
    df['Post_Body_EN'] = df['Post_Body_EN'].fillna('').astype(str)
    df['fre'] = df['Post_Body_EN'].apply(textstat.flesch_reading_ease)
    return df

def calculate_fk_grade(df):
    df['Post_Body_EN'] = df['Post_Body_EN'].fillna('').astype(str)
    df['fk_grade'] = df['Post_Body_EN'].apply(textstat.flesch_kincaid_grade)
    return df

def calculate_gunning_fog(df):
    df['Post_Body_EN'] = df['Post_Body_EN'].fillna('').astype(str)
    df['gunning_fog'] = df['Post_Body_EN'].apply(textstat.gunning_fog)
    return df

def calculate_smog_index(df):
    df['Post_Body_EN'] = df['Post_Body_EN'].fillna('').astype(str)
    df['smog_index'] = df['Post_Body_EN'].apply(textstat.smog_index)
    return df

def calculate_coleman_liau_index(df):
    df['Post_Body_EN'] = df['Post_Body_EN'].fillna('').astype(str)
    df['coleman_liau_index'] = df['Post_Body_EN'].apply(textstat.coleman_liau_index)
    return df

def calculate_lix(df):
    df['Post_Body_EN'] = df['Post_Body_EN'].fillna('').astype(str)
    df['lix'] = df['Post_Body_EN'].apply(calculate_lix_text)
    return df

def calculate_rix(df):
    df['Post_Body_EN'] = df['Post_Body_EN'].fillna('').astype(str)
    df['rix'] = df['Post_Body_EN'].apply(calculate_rix_text)
    return df





In [66]:
from tqdm import tqdm

# List of all the functions
functions = [calculate_ari, calculate_fre, calculate_fk_grade, calculate_gunning_fog, 
             calculate_smog_index, calculate_coleman_liau_index, calculate_lix, calculate_rix]

# Iterate over functions with progress bar
for func in tqdm(functions):
    df = func(df)


100%|██████████| 8/8 [55:38<00:00, 417.37s/it]


Check the results

In [58]:
# for key in dfr.keys():
#     dfr[key].to_csv(f'Master Thesis Data\\dfr\\{key}.csv', index=False)


In [ ]:
# #Save the results as:
# df_combined.to_csv('Master Thesis Data\\df_clean_final.csv', index=False)

In [69]:
#Save the results as:
df.to_csv('Master Thesis Data\\df_clean_final.csv', index=False)

In [57]:
end_time = time.time()
total_time = end_time - start_time
print("Total execution time: {:.2f} seconds".format(total_time))

Total execution time: 6216.41 seconds
